In [ ]:
import os
import sys
import rarfile
import py7zr
import pandas as pd
from pynxtools_apm.utils.versioning import __version__
from pynxtools_apm.examples.get_sha256_of_directories import (
    analyze_file,
    analyze_zip_file,
    analyze_tar_file,
    analyze_rar_file,
    analyze_sevenzip_file,
    analyze_zip_file,
    analyze_tar_file,
    SEPARATOR,
)
from pynxtools_apm.examples.oasisb_utils import CSV_HEADER_FOR_HASH_FILE

with open("source_directory.txt") as fp:
    src_directory = f"{fp.read().strip().replace('/', os.sep)}"
print(src_directory)

In [ ]:
config: dict[str, str] = {
    "python_version": f"{sys.version.replace(' ', '_')}",
    "working_directory": f"{os.getcwd()}",
    "pynxtools_apm version": f"{__version__}",
    "rarfile version": f"{rarfile.__version__}",
    "sevenzip version": f"{py7zr.__version__}",
    # "blake3-py version": f"{blake3.__version__}",
    # "blake3-py max_threads": f"{blake3.blake3.AUTO}",
    # "directory": f"src_directory,  # sys.argv[1],
}

spread_sheet_of_all_projects = pd.read_excel(
    f"{src_directory}{os.sep}aaa_legacy_data.ods",
    sheet_name="aaa_legacy_data",
    engine="odf",
)
# cnt = 0
for row in spread_sheet_of_all_projects.itertuples(index=True):
    if row.project_name not in ("deu_stuttgart_eich_tap"):
        continue
    # if not row.project_name.startswith("usa_idaho_boise"):
    #     continue
    if row.parse in (1, 2):
        print(f"project{SEPARATOR}{row.project_name}{SEPARATOR}hashing...")
        sub_directory = f"{src_directory}{os.sep}{row.project_name}"
        results = []
        issues = []
        project_config = config
        prefix = f"{sub_directory}{os.sep}"
        project_config["directory"] = prefix
        for key, value in project_config.items():
            results.append(f"{key}{SEPARATOR}{value}")
            issues.append(f"{key}{SEPARATOR}{value}")
        del project_config, key, value
        results.append(CSV_HEADER_FOR_HASH_FILE)
        for root, dirs, files in os.walk(sub_directory):
            for file in files:
                fpath = f"{root}/{file}".replace(os.sep * 2, os.sep)
                # fname = os.path.basename(fpath)
                suffix = fpath.replace(config["directory"], "")

                if fpath.lower().endswith((".zip", ".eln")):
                    analyze_zip_file(fpath, results, issues, prefix)
                elif fpath.lower().endswith((".tar", ".tar.gz", ".tar.bz2", ".tar.xz")):
                    analyze_tar_file(fpath, results, issues, prefix)
                elif fpath.lower().endswith((".rar")):
                    analyze_rar_file(fpath, results, issues, prefix)
                elif fpath.lower().endswith((".7z")):
                    analyze_sevenzip_file(fpath, results, issues, prefix)
                else:
                    analyze_file(fpath, results, issues, prefix)
                # cnt += 1
        del root, dirs, files, file, fpath, suffix
        # for entry in results:
        #     print(entry)
        with open(
            f"{src_directory}{os.sep}{row.project_name}.sha256.results.csv",
            "w",
            encoding="utf-8",
            errors="surrogateescape",
        ) as fp:
            fp.write("\n".join(results))
        del results
        with open(
            f"{src_directory}{os.sep}{row.project_name}.sha256.issues.csv",
            "w",
            encoding="utf-8",
            errors="surrogateescape",
        ) as fp:
            fp.write("\n".join(issues))
        if len(issues) > 6:
            print(issues)
        else:
            print(
                f"project{SEPARATOR}{row.project_name}{SEPARATOR}no issues"
            )  #  with {cnt} paths")
        del issues, sub_directory, prefix
        # cnt = 0